In [ ]:
from pycocotools.coco import COCO
import requests
import csv
import concurrent.futures
from pascal_voc_writer import Writer

In [ ]:
coco = COCO('instances_train2017.json')

In [ ]:
# Let's see all the classes available in the COCO dataset
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

In [ ]:
# Let's say we are interested only in the 'person' class
catIds = coco.getCatIds(catNms=['person'])  # Here you may replace person with any other class that you want from the COCO classes
imgIds = coco.getImgIds(catIds=catIds )
images = coco.loadImgs(imgIds)

In [ ]:
# Defining the function to download the images
def image_downloader(im):
    print("im: ", im)
    img_data = requests.get(im['coco_url']).content
    with open('class/' + im['file_name'], 'wb') as handler:
        handler.write(img_data)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(image_downloader, images)

In [ ]:
with open('annotations_' + 'person' + '.csv', mode='w', newline='') as annot:
    for im in images:
        annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        for i in range(len(anns)):
            annot_writer = csv.writer(annot)
            annot_writer.writerow(['class/' + im['file_name'], int(round(anns[i]['bbox'][0])), int(round(anns[i]['bbox'][1])), int(round(anns[i]['bbox'][0] + anns[i]['bbox'][2])), int(round(anns[i]['bbox'][1] + anns[i]['bbox'][3])),im['width'],
im['height'], 'person'])
    annot.close()